In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [16]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge

In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
from sklearn.preprocessing import PolynomialFeatures

In [19]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import geopandas as gpd
import shapely

In [21]:
neighbor = gpd.GeoDataFrame.from_file('../geo/neighborhoods.geojson')
comm = pd.read_csv('../geo/zip_comm.csv')
census = pd.read_csv('../geo/census_zip.csv')

In [22]:
comm['ZCTA5'] = comm['ZCTA5'].astype('int').astype('str')
comm['CHGOCA'] = comm['CHGOCA'].astype('int').astype('str')

In [23]:
census['zip'] = census['zip'].astype('int').astype('str')
census['tract'] = census['tract'].astype('str')

In [24]:
data = ['EMSOD','HCSBDP','HCSBHP','HCSNVP',
             'HCSSPDP','HDBHR','HDDRR','HDSZR',
        'MEODR','YRDEP','VRALR','VRDIDR', 'VRDOR', 'VRSUR']

In [25]:
files = [f'../data/{file}.csv' for file in data]

In [26]:
mh = pd.concat(map(pd.read_csv, files),axis = 0)
mh = mh[mh['p'].isnull()]
mh = mh[mh['l'] != 'place']
mh['g'] = mh['g'].astype('str')

mh = mh.merge(comm, how = 'left', left_on = 'g', right_on = 'ZCTA5')

In [27]:
mh.loc[mh['g'].str.len() == 5,'g'] = mh['CHGOCA']
mh['g'] = mh['g'].str.replace('1714000-','')

In [28]:
neighbor = neighbor.merge(mh, how = 'left', left_on = 'area_numbe', right_on = 'g')

In [29]:
neighbor['long'] = neighbor['geometry'].centroid.x
neighbor['lat'] = neighbor['geometry'].centroid.y

C:\Users\Finley & Stein\AppData\Local\Temp\ipykernel_4492\498829464.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighbor['long'] = neighbor['geometry'].centroid.x
C:\Users\Finley & Stein\AppData\Local\Temp\ipykernel_4492\498829464.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighbor['lat'] = neighbor['geometry'].centroid.y


----

##  CZA

In [180]:
CZA = pd.read_csv(f'../data/CZA.csv')
CZB = pd.read_csv(f'../data/CZB.csv')
CZD = pd.read_csv(f'../data/CZD.csv')
CZH = pd.read_csv(f'../data/CZH.csv')
CZL = pd.read_csv(f'../data/CZL.csv')
CZM = pd.read_csv(f'../data/CZM.csv')
CZO = pd.read_csv(f'../data/CZO.csv')
CZP = pd.read_csv(f'../data/CZP.csv')
CZR = pd.read_csv(f'../data/CZR.csv')
CZS = pd.read_csv(f'../data/CZS.csv')
CZT = pd.read_csv(f'../data/CZT.csv')
CZU = pd.read_csv(f'../data/CZU.csv')
CZV = pd.read_csv(f'../data/CZV.csv')

In [195]:
def transform(df):
    df.drop(df[df['p'].notnull()].index, inplace = True)
    df.drop(df[df['l'] == 'place'].index, inplace = True)

    df = df.merge(census, how = 'left', left_on = 'g', right_on = 'tract')
    df['zip'] = df['zip'].astype('str')
    df.loc[df['l'] == 'tract-2010','g'] = df['zip']

    df = df.merge(comm, how = 'left', left_on = 'g', right_on = 'ZCTA5')
    df['zip'] = df['zip'].astype('str')
    df.loc[df['l'] == 'tract-2010','g'] = df['zip']

    df.loc[df['g'].str.len() == 5,'g'] = df['CHGOCA']
    df['g'] = df['g'].str.replace('1714000-','')

    return df.groupby('g')['v'].agg('mean')



In [202]:
pd.DataFrame(neighbor.groupby(['area_numbe','a'])['v'].agg('mean'))

v
area_numbe a                 
1          EMSOD    84.628889
           HCSBDP   37.199517
           HCSBHP   51.152374
           HCSNVP   38.256938
           HCSSPDP  16.615559
...                       ...
9          MEODR    11.140000
           VRALR     9.450000
           VRDIDR   13.950000
           VRDOR    13.975000
           VRSUR    11.100000

[944 rows x 1 columns]

In [197]:
pd.DataFrame(transform(CZA))

,v
g,
1,65.690897
10,20.764998
11,25.145009
12,28.879218
13,35.954990
...,...
75,153.449613
76,22.899384
77,50.792556


In [191]:
def merge(df):
    return transform(df).merge(neighbor, how = 'outer', on = ['g','d'])
    # df.rename(columns = {'a_y':'mh_stat','v_y':'mh_val','v_x':'crim_val'}, inplace = True)
    

In [194]:
transform(CZA).head()

,Unnamed: 0,se,g,l,a,p,d,v,zip,tract,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio,ZCTA5,CHGOCA,TOT2010
0,0,NaN,32,zip,CZA,NaN,2018-2022,66.454485,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60601,32,11110.0
1,1,NaN,32,zip,CZA,NaN,2018-2022,11.133545,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60602,32,1204.0
2,2,NaN,32,zip,CZA,NaN,2018-2022,10.292674,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60603,32,493.0
3,3,NaN,32,zip,CZA,NaN,2018-2022,7.319068,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60604,32,570.0
4,4,NaN,28,zip,CZA,NaN,2018-2022,22.908756,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60606,28,1783.0


In [193]:
merge(CZD).isnull().sum()

Unnamed: 0_x             6662
se_x                    89356
g                         581
l_x                      6662
a_x                      6662
p_x                    156038
d                           0
v_x                      6662
zip                      6662
tract                   15062
usps_zip_pref_city      15062
usps_zip_pref_state     15062
res_ratio               15062
bus_ratio               15062
oth_ratio               15062
tot_ratio               15062
ZCTA5_x                  9399
CHGOCA_x                 9399
TOT2010_x                9399
community              103404
area                   103404
shape_area             103404
perimeter              103404
area_num_1             103404
area_numbe             103404
comarea_id             103404
comarea                103404
shape_len              103404
geometry               103404
Unnamed: 0_y           103404
se_y                   104204
l_y                    103404
a_y                    103404
p_y       

In [170]:
X = merge(CZA)
X = X[X['mh_stat'].notnull()]

In [168]:
values = []
fvalue_countsin [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
    X = merge(stat)
    X = X[X['mh_stat'].notnull()]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    print(lr.coef_)

[   0.30681033 -137.12567954    7.35475115    3.84357762   -4.18063291
   -4.46136261    4.79841791]
[ 1.64916722e-02 -6.83041055e+01 -3.78455308e+00  1.80135135e-01
 -2.27897210e-01 -2.32181064e-01  2.79943139e-01]
[   0.69582226  -66.47869581 -105.64791055    8.80142162   -9.53053711
  -10.14175659   10.87087208]
[ 0.01932033 -8.83224436 -1.91199222  0.24227373 -0.26183517 -0.28098911
  0.30055054]
[ 3.26213528e-02  1.56166652e+02  4.55980954e+02  2.74416967e-01
 -6.27699952e-01 -3.50696590e-01  7.03979575e-01]
[  0.49070733 -20.39955355 371.06152046   5.0575787   -6.14543006
  -6.6166459    7.70449726]
[ 0.00767931 -4.89183087 -3.12890021  0.09500158 -0.1026471  -0.11186384
  0.11950937]
[ 2.24595424e-01  2.75055529e+01  4.37227103e+02  3.27117599e+00
 -2.34536235e+00 -3.85658426e+00  2.93077062e+00]
[  0.15865301 -43.80798676  22.13629718   1.99811064  -2.16630634
  -2.32251784   2.49071354]
[ 0.01795676 -5.40341429  5.36932959  0.22649561 -0.24882028 -0.26349336
  0.28581804]
[  0

In [120]:
X['mh_stat'].values

KeyError: 'mh_stat'

In [109]:
values = []
for stat in [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
    X = transform(stat)
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
        
    base = [np.mean(y_train)] * len(y_test)
    baseline = mean_squared_error(y_test, base, squared=False)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    lr_train = mean_squared_error(y_train, lr.predict(X_train), squared=False)
    lr_test = mean_squared_error(y_test, lr.predict(X_test), squared=False)
    print(('lr',lr_train,lr_test))
    
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    lasso_train = mean_squared_error(y_train,lasso.predict(X_train), squared = False)
    lasso_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)
    print(('lasso',lasso_train, lasso_test))
    
    rg = Ridge()
    rg.fit(X_train, y_train)
    rg_train = mean_squared_error(y_train,rg.predict(X_train), squared = False) 
    rg_test = mean_squared_error(y_test, rg.predict(X_test), squared=False)
    print(('rg',rg_train,rg_test))
    
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    knn_train = mean_squared_error(y_train,knn.predict(X_train), squared = False)
    knn_test = mean_squared_error(y_test, knn.predict(X_test), squared=False)
    print(('knn',knn_train,knn_test))

    dt = DecisionTreeRegressor()
    dt.fit(X_train, y_train)
    dt_train = mean_squared_error(y_train,dt.predict(X_train), squared = False) 
    dt_test = mean_squared_error(y_test, dt.predict(X_test), squared=False)
    print(('dt',dt_train,dt_test))

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    rf_train = mean_squared_error(y_train,rf.predict(X_train), squared = False)
    rf_test = mean_squared_error(y_test, rf.predict(X_test), squared=False)
    print(('rf',rf_train,rf_test))

    abr = AdaBoostRegressor()
    abr.fit(X_train, y_train)
    abr_train = mean_squared_error(y_train,abr.predict(X_train), squared = False)
    abr_test = mean_squared_error(y_test, abr.predict(X_test), squared=False)
    print(('abr',abr_train,abr_test))
    values.append([baseline, lr_train, lr_test, lasso_train, lasso_test, rg_train, rg_test, knn_train, knn_test, dt_train, dt_test, rf_train, rf_test, abr_train, abr_test])

('lr', 11.3600070079458, 11.329621979236302)
('lasso', 11.767633355039608, 11.757083249309828)
('rg', 11.359377356665497, 11.32959911340376)
('knn', 1.530250627707911e-15, 1.5060851752206681e-15)
('dt', 5.703017116235093e-14, 5.670540184453513e-14)
('rf', 0.00014267800255386996, 0.0002471255495457448)
('abr', 6.450957461766333, 6.41229911796659)
('lr', 6.4679987418509866, 6.518705365772849)
('lasso', 6.555697083668511, 6.600462334832983)
('rg', 6.467981145275853, 6.518816173358741)
('knn', 7.566608288108627e-16, 7.525981095821164e-16)
('dt', 4.3135500545029845e-14, 4.190664272870676e-14)
('rf', 0.0004655433527164512, 0.0005701718336497516)
('abr', 3.2052154168269924, 3.167383466975646)
('lr', 23.14281497234884, 23.24668099050682)
('lasso', 23.41537115051448, 23.545532189595356)
('rg', 23.141888353077615, 23.242907397567258)
('knn', 2.1254581338350098e-15, 2.1878692845711333e-15)
('dt', 6.17790594705991e-14, 6.13243813014502e-14)
('rf', 5.386335410557381e-14, 5.328364350055871e-14)
('ab

In [ ]:
, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV

In [92]:
for stat in [CZB]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    print(pd.DataFrame([lr.coef_,X.columns]))

         0         1         2                     3                     4  \
0  0.38425 -6.187779 -0.233534  3041353290348.381348  3031488385801.487305   
1   mh_val       lat      long         mh_stat_VRALR        mh_stat_VRDIDR   

                      5                     6  
0  3032787520715.342285  3040419290669.819336  
1         mh_stat_VRDOR         mh_stat_VRSUR  


In [110]:
errors = pd.DataFrame(values, columns = ['baseline', 'lr_train', 'lr_test', 'lasso_train', 'lasso_test', 
                                'rg_train', 'rg_test', 'knn_train', 'knn_test', 
                                'dt_train', 'dt_test', 'rf_train', 'rf_test', 
                                'abr_train', 'abr_test'])

In [111]:
columns = pd.DataFrame(['CZA','CZB', 'CZD', 'CZH', 'CZL','CZM', 'CZO', 'CZP', 'CZR', 'CZS', 'CZT', 'CZV'])

In [112]:
errors = pd.concat([errors,columns], axis = 1)

In [113]:
errors.set_index(0, inplace = True)

In [114]:
errors.to_csv('../data/errors.csv')

In [115]:
errors.describe()

,baseline,lr_train,lr_test,lasso_train,lasso_test,rg_train,rg_test,knn_train,knn_test,dt_train,dt_test,rf_train,rf_test,abr_train,abr_test
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,12.000000,12.000000
mean,28.990317,25.940125,25.447681,26.169663,25.685439,25.939223,25.445374,3.739697e-15,3.691298e-15,1.506711e-13,1.519678e-13,1.782623e-04,2.366680e-04,9.009228,8.960276
std,32.644718,32.453803,31.705664,32.344012,31.592465,32.452511,31.702394,5.039662e-15,5.017766e-15,2.135999e-13,2.169749e-13,4.149824e-04,5.403029e-04,9.791107,9.763313
min,0.528688,0.320516,0.319941,0.527336,0.528688,0.320509,0.319954,5.099318e-17,5.138775e-17,1.975861e-15,1.944802e-15,2.705038e-15,2.670506e-15,0.209126,0.210170
25%,6.196792,4.592792,4.621457,4.893647,4.933097,4.592778,4.621613,6.035786e-16,5.984662e-16,2.910729e-14,2.845068e-14,9.403638e-14,9.427610e-14,2.136363,2.142493
50%,14.331292,9.487173,9.455224,9.862044,9.853818,9.486841,9.455222,1.243998e-15,1.229857e-15,5.008284e-14,4.930602e-14,1.205243e-06,2.087542e-06,4.828086,4.789841
75%,40.757704,35.407579,34.779581,35.621511,35.005133,35.406884,34.776738,4.848978e-15,4.843154e-15,1.868589e-13,1.878329e-13,9.802281e-05,1.499623e-04,14.414673,14.225703
max,88.575164,83.456285,84.362163,83.507619,84.444862,83.448501,84.355678,1.496712e-14,1.574919e-14,6.188427e-13,6.322591e-13,1.425213e-03,1.866042e-03,25.444888,25.098484


In [78]:
#for stat in [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
 #   X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
  #  avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
   # X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
#    y = X['crim_val']
 #   X = X.drop(columns = ['crim_val','d'])
    
  #  X = pd.get_dummies(X, columns = ['mh_stat'])
    
   # X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    #ss = StandardScaler()
    #X_train = ss.fit_transform(X_train)
    #X_test = ss.transform(X_test)
    
#    gs = GridSearchCV(DecisionTreeRegressor(),
 #                     param_grid = {'max_depth':[3],
  #                                 'min_samples_leaf':[3,5,7,10]},
   #                   cv = 5,
    #                  n_jobs = -1)
    
 #   gs.fit(X_train, y_train)
  #  print(gs.best_params_)

In [ ]:
, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV

In [69]:
from sklearn.tree import export_text

In [81]:
for stat in [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    dt = DecisionTreeRegressor(max_depth=3, min_samples_leaf=3)
    dt.fit(X_train,y_train)
    print(export_text(dt))

|--- feature_1 <= 0.51
|   |--- feature_1 <= -0.83
|   |   |--- feature_2 <= 0.19
|   |   |   |--- value: [41.04]
|   |   |--- feature_2 >  0.19
|   |   |   |--- value: [57.81]
|   |--- feature_1 >  -0.83
|   |   |--- feature_1 <= 0.05
|   |   |   |--- value: [27.66]
|   |   |--- feature_1 >  0.05
|   |   |   |--- value: [43.89]
|--- feature_1 >  0.51
|   |--- feature_1 <= 0.64
|   |   |--- feature_0 <= 0.22
|   |   |   |--- value: [25.19]
|   |   |--- feature_0 >  0.22
|   |   |   |--- value: [25.08]
|   |--- feature_1 >  0.64
|   |   |--- feature_1 <= 0.96
|   |   |   |--- value: [14.20]
|   |   |--- feature_1 >  0.96
|   |   |   |--- value: [10.66]

|--- feature_1 <= -0.76
|   |--- feature_1 <= -1.70
|   |   |--- feature_1 <= -1.79
|   |   |   |--- value: [31.91]
|   |   |--- feature_1 >  -1.79
|   |   |   |--- value: [10.41]
|   |--- feature_1 >  -1.70
|   |   |--- feature_2 <= 0.76
|   |   |   |--- value: [35.75]
|   |   |--- feature_2 >  0.76
|   |   |   |--- value: [39.72]
|--- 

In [223]:
[X_test,y_pred]

[array([[ 1.        , -0.72260815, -1.51503973, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.091508  , -0.48239101, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.4717863 ,  1.03157575, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        ...,
        [ 1.        ,  2.837444  ,  0.4378657 , ...,  3.01478808,
         -1.0050396 ,  0.33504995],
        [ 1.        , -0.3544664 ,  1.23083567, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.86824665, -0.27181914, ...,  0.33169827,
         -0.99498567,  2.98462961]]),
 array([50.63247933, 18.938757  ,  7.79390601, ..., 63.73305938,
        13.3781609 , 23.45584757])]